In [1]:
%reload_ext autoreload
%autoreload 2

In [1]:
from padding_oracle import CBCPaddingOracle
import requests
from copy import copy

# Target addresses
token_addr = "https://cbc-rsa.syssec.dk:8000/"
oracle_addr = "https://cbc-rsa.syssec.dk:8000/quote"

def oracle(iv: bytes, block: bytes):
    # Create token with the iv and block
    token = iv + block

    # Create request and attach cookie with token
    cookie = {"authtoken": token.hex()}
    r = requests.get(oracle_addr, cookies=cookie)
    
    # Check response against some known error responses.
    if (r.content == b'Padding is incorrect.' or r.content == b'PKCS#7 padding is incorrect.' or  r.content == b'Zero-length input cannot be unpadded'): #r.content == b'No quote for you!' or
        return False
    else:
        return True

# Establish baseline ciphertext
r = requests.get(token_addr)
ciphertext_str = r.cookies["authtoken"]
ciphertext_bytes_baseline = bytearray.fromhex(ciphertext_str)

# Find block size by changing bytes from the iv end of the ciphertext until we get a padding error. 
# We know that the ciphertext contains an IV in the front.
ciphertext_bytes = copy(ciphertext_bytes_baseline)
block_size = 0
for i in range(len(ciphertext_bytes)):
    ciphertext_bytes[i] ^= 0x25
    r = requests.get(oracle_addr, cookies={"authtoken": ciphertext_bytes.hex()})
    if (r.content != b'No quote for you!'):
        break
    else:
        block_size += 1

# Break encryption using the CBC Padding Oracle
breaker = CBCPaddingOracle(copy(ciphertext_bytes_baseline), block_size, oracle)
result = breaker.attack()

CBC-Padding-Oracle Attack
-------------------------
- IV + Ciphertext: 9d462d97f0aed2d7b904694414871fde304a6c28e46344ab2c7348e814cabb81e763b69f9b80c9ad50e11157186b8c16d60e233e76a7d2ff08036efe114cd6e3bae4a5b2dfe62cf3cdac8b5b71680784ebaf86f8c88ca3ed1d60e739f6eadaf4a3d47e10f8ee9a52b2f7a8fd0d3b27c4
- Block size: 16
- Block count: 7
-------------------------
Performing attack...
-------------------------
Block 0 / IV:      9d462d97f0aed2d7b904694414871fde
------------------------------------------------------------
Block 1:           d43948a78edbb4a2db341f3d63e27dab                                                                                                                                                                         
DEC-Block 1:       c42958b79ecba4b2cb240f2d73f26dbb
Plaintext-Block 1: 596f75206e6576657220666967757265
------------------------------------------------------------
Block 2:           0035094cd4073cda48437ab124a9c3fe                                               